# Лабораторная работа 1

### Алгоритм Кадане

Алгоритм Кадане решает задачу максимальной суммы непрерывного подмассива за один проход по массиву:

In [1]:
import random
import time
import math
import psutil
import statistics
import tracemalloc
from dataclasses import dataclass

In [ ]:
# наивная реализация
def max_subarray_kadane(a):
    cur = a[0] # макс. сумма подмассива, оканчивающегося в текущем элементе
    best = a[0] # макс. сумма среди всех подмассивов, которые мы видели до текущего элемента
    for x in a[1:]:
        cur = max(x, cur + x) # либо продолжаем текущий подмассив, либо начинаем новый с элемента x
        best = max(best, cur) # обновляем лучший результат
    return best

*Сложность*: $O(n)$ по времени (проход по массиву) и $O(n)$ по памяти (создание временного массива)

In [ ]:
# оптимизированная
def max_subarray_kadane_fast(a):
    n = len(a)
    cur = a[0]
    best = cur
    
    for i in range(1, n): # избегаем создание временного списка
        x = a[i]
        sum = cur + x
        # заменили вызов функции на ветвление
        cur = x if x > sum else sum
        best = cur if cur > best else best

    return best

*Сложность*: $O(n)$ по времени (проход по массиву) и $O(1)$ по памяти

In [4]:
tests = [
    ([-2, 1, -3, 4, -1, 2, 1, -5, 4], 6),
    ([1, 2, 3], 6),
    ([-1, -2, -3], -1),
    ([5, -2, 3, -1, 2, -10, 10], 10),
    ([0, 0, 0], 0),
]

for arr, ans in tests:
    r2 = max_subarray_kadane(arr)
    r3 = max_subarray_kadane_fast(arr)
    print(arr, "=>", r2, r3, "OK" if (r2==ans and r3==ans) else "FAIL")

[-2, 1, -3, 4, -1, 2, 1, -5, 4] => 6 6 OK
[1, 2, 3] => 6 6 OK
[-1, -2, -3] => -1 -1 OK
[5, -2, 3, -1, 2, -10, 10] => 10 10 OK
[0, 0, 0] => 0 0 OK


In [5]:
@dataclass
class BenchResult:
    n: int
    seconds: float
    peak_tracemalloc_mb: float
    rss_mb: float | None

In [6]:
def bench(func, arr, repeats=5):
    # прогрев
    func(arr)

    times = []
    peaks = []
    rss_vals = []

    for _ in range(repeats):
        tracemalloc.start()
        rss_before = psutil.Process().memory_info().rss / (1024**2)

        t0 = time.perf_counter()
        func(arr)
        t1 = time.perf_counter()

        _, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()

        rss_after = psutil.Process().memory_info().rss / (1024**2)

        times.append(t1 - t0)
        peaks.append(peak / (1024**2))
        if rss_before is not None and rss_after is not None:
            rss_vals.append(max(rss_before, rss_after))

    return BenchResult(
        n=len(arr),
        seconds=statistics.median(times),
        peak_tracemalloc_mb=statistics.median(peaks),
        rss_mb=statistics.median(rss_vals) if rss_vals else None
    )
    
    
def make_array(n, seed=42, lo=-100, hi=100):
    rnd = random.Random(seed)
    return [rnd.randint(lo, hi) for _ in range(n)]

In [7]:
def run_benchmarks():
    n1_list = [10_000, 100_000, 300_000, 1_000_000]

    results = {"kadane": [], "kadane_fast": []}

    for n in n1_list:
        arr = make_array(n, seed=42)
        res2 = bench(max_subarray_kadane, arr, repeats=5)
        res3 = bench(max_subarray_kadane_fast, arr, repeats=5)
        results["kadane"].append(res2)
        results["kadane_fast"].append(res3)

    return results

results = run_benchmarks()

In [9]:
def print_table(title, rows):
    print("\n" + title)
    print("-" * len(title))
    print(f"{'n':>10} | {'time, s (median)':>16} | {'peak tracemalloc, MB':>20} | {'RSS MB':>10}")
    print("-" * 70)
    for r in rows:
        rss = f"{r.rss_mb:.1f}" if r.rss_mb is not None else "N/A"
        print(f"{r.n:>10} | {r.seconds:>16.6f} | {r.peak_tracemalloc_mb:>20.3f} | {rss:>10}")

print_table("Kadane", results["kadane"])
print_table("Kadane fast", results["kadane_fast"])


Kadane
------
         n | time, s (median) | peak tracemalloc, MB |     RSS MB
----------------------------------------------------------------------
     10000 |         0.021893 |                0.076 |       70.1
    100000 |         0.177775 |                0.763 |       73.7
    300000 |         0.496742 |                2.289 |       78.1
   1000000 |         1.664845 |                7.630 |       95.5

Kadane fast
-----------
         n | time, s (median) | peak tracemalloc, MB |     RSS MB
----------------------------------------------------------------------
     10000 |         0.009845 |                0.001 |       70.1
    100000 |         0.099704 |                0.001 |       73.7
    300000 |         0.297121 |                0.001 |       78.1
   1000000 |         0.947130 |                0.001 |       94.3
